2. Простой пример: кроссплатформенный GUI
Задача: приложение должно работать под Windows и macOS, отображая элементы управления в «родном» стиле.

In [1]:
from abc import ABC, abstractmethod

# ---------- Abstract Products ----------
class Button(ABC):
    @abstractmethod
    def paint(self) -> None: ...

class Checkbox(ABC):
    @abstractmethod
    def paint(self) -> None: ...

# ---------- Concrete Products ----------
class WinButton(Button):
    def paint(self) -> None:
        print("🪟 Рисуем кнопку Windows‑style")

class MacButton(Button):
    def paint(self) -> None:
        print("🍎 Рисуем кнопку macOS‑style")

class WinCheckbox(Checkbox):
    def paint(self) -> None:
        print("🪟 Рисуем чекбокс Windows‑style")

class MacCheckbox(Checkbox):
    def paint(self) -> None:
        print("🍎 Рисуем чекбокс macOS‑style")

# ---------- Abstract Factory ----------
class GUIFactory(ABC):
    @abstractmethod
    def create_button(self) -> Button: ...
    @abstractmethod
    def create_checkbox(self) -> Checkbox: ...

# ---------- Concrete Factories ----------
class WinFactory(GUIFactory):
    def create_button(self) -> Button:
        return WinButton()
    def create_checkbox(self) -> Checkbox:
        return WinCheckbox()

class MacFactory(GUIFactory):
    def create_button(self) -> Button:
        return MacButton()
    def create_checkbox(self) -> Checkbox:
        return MacCheckbox()

# ---------- Client ----------
class Application:
    def __init__(self, factory: GUIFactory) -> None:
        self._button = factory.create_button()
        self._checkbox = factory.create_checkbox()

    def render(self) -> None:
        self._button.paint()
        self._checkbox.paint()

# ---------- Использование ----------
if __name__ == "__main__":
    for factory in (WinFactory(), MacFactory()):
        app = Application(factory)
        app.render()


🪟 Рисуем кнопку Windows‑style
🪟 Рисуем чекбокс Windows‑style
🍎 Рисуем кнопку macOS‑style
🍎 Рисуем чекбокс macOS‑style


3. Сложный пример: мульти‑облачная инфраструктура
Сценарий: DevOps‑платформа должна уметь поднимать виртуальные машины, базы данных и балансировщики в AWS, Azure и GCP.
Хотим легко подключать новые провайдеры (DigitalOcean, Yandex Cloud…) без изменения кода оркестратора.



In [2]:
"""
Мульти‑облачная фабрика: создаём согласованный стек ресурсов в разных облаках.
"""
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Dict, Type

# ---------- Abstract Products ----------
class VM(ABC):
    @abstractmethod
    def start(self) -> None: ...

class DB(ABC):
    @abstractmethod
    def backup(self) -> None: ...

class LB(ABC):
    @abstractmethod
    def attach(self, vm: VM) -> None: ...

# ---------- Concrete Products: AWS ----------
class AWSVM(VM):
    def start(self) -> None:
        print("🚀 EC2 instance запущена")

class AWSDB(DB):
    def backup(self) -> None:
        print("💾 Снимок RDS создан")

class AWSLB(LB):
    def attach(self, vm: VM) -> None:
        print("🔗 EC2 прикреплена к ELB")

# ---------- Concrete Products: Azure ----------
class AzureVM(VM):
    def start(self) -> None:
        print("🚀 Azure VM запущена")

class AzureDB(DB):
    def backup(self) -> None:
        print("💾 Бэкап Azure SQL готов")

class AzureLB(LB):
    def attach(self, vm: VM) -> None:
        print("🔗 VM прикреплена к Azure Load Balancer")

# ---------- Concrete Products: GCP ----------
class GCPVM(VM):
    def start(self) -> None:
        print("🚀 Compute Engine VM запущена")

class GCPDB(DB):
    def backup(self) -> None:
        print("💾 Бэкап Cloud SQL готов")

class GCPLB(LB):
    def attach(self, vm: VM) -> None:
        print("🔗 VM прикреплена к GCP Load Balancer")

# ---------- Abstract Factory ----------
class CloudFactory(ABC):
    @abstractmethod
    def create_vm(self) -> VM: ...
    @abstractmethod
    def create_db(self) -> DB: ...
    @abstractmethod
    def create_lb(self) -> LB: ...

# ---------- Concrete Factories ----------
class AWSFactory(CloudFactory):
    def create_vm(self) -> VM: return AWSVM()
    def create_db(self) -> DB: return AWSDB()
    def create_lb(self) -> LB: return AWSLB()

class AzureFactory(CloudFactory):
    def create_vm(self) -> VM: return AzureVM()
    def create_db(self) -> DB: return AzureDB()
    def create_lb(self) -> LB: return AzureLB()

class GCPFactory(CloudFactory):
    def create_vm(self) -> VM: return GCPVM()
    def create_db(self) -> DB: return GCPDB()
    def create_lb(self) -> LB: return GCPLB()

# ---------- Реестр фабрик (плагин‑трюк) ----------
_FACTORY_REGISTRY: Dict[str, Type[CloudFactory]] = {
    "aws": AWSFactory,
    "azure": AzureFactory,
    "gcp": GCPFactory,
}

def register_cloud(name: str):
    """Декоратор для сторонних облаков."""
    def decorator(cls: Type[CloudFactory]):
        _FACTORY_REGISTRY[name] = cls
        return cls
    return decorator

# ---------- Клиент ----------
class Orchestrator:
    def __init__(self, cloud: str) -> None:
        self._factory = _FACTORY_REGISTRY[cloud.lower()]()

    def provision_stack(self) -> None:
        vm = self._factory.create_vm()
        db = self._factory.create_db()
        lb = self._factory.create_lb()

        vm.start()
        db.backup()
        lb.attach(vm)

# ---------- Демонстрация ----------
if __name__ == "__main__":
    for cloud in ("aws", "azure", "gcp"):
        print(f"\n=== Provisioning in {cloud.upper()} ===")
        Orchestrator(cloud).provision_stack()

    # Добавляем новое облако «Yandex Cloud» без изменения Orchestrator
    @register_cloud("yc")
    class YCFactory(CloudFactory):
        class YCVM(VM):
            def start(self): print("🚀 YC VM запущена")
        class YCDB(DB):
            def backup(self): print("💾 Бэкап YC PG готов")
        class YCLB(LB):
            def attach(self, vm): print("🔗 VM прикреплена к YC LB")
        def create_vm(self): return self.YCVM()
        def create_db(self): return self.YCDB()
        def create_lb(self): return self.YCLB()

    print("\n=== Provisioning in YC ===")
    Orchestrator("yc").provision_stack()



=== Provisioning in AWS ===
🚀 EC2 instance запущена
💾 Снимок RDS создан
🔗 EC2 прикреплена к ELB

=== Provisioning in AZURE ===
🚀 Azure VM запущена
💾 Бэкап Azure SQL готов
🔗 VM прикреплена к Azure Load Balancer

=== Provisioning in GCP ===
🚀 Compute Engine VM запущена
💾 Бэкап Cloud SQL готов
🔗 VM прикреплена к GCP Load Balancer

=== Provisioning in YC ===
🚀 YC VM запущена
💾 Бэкап YC PG готов
🔗 VM прикреплена к YC LB
